**1. Required dependecies**

In [2]:
%pip install -U datasets==2.17.0

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 --quiet #hugging face

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.

**2. Load the datasets, Large Language Model (LLM), tokenizer, and configurator.**

In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [4]:
huggingface_dataset_name = "samsum" # already created dataset

dataset = load_dataset(huggingface_dataset_name)

example_indices = [50, 100]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices): # examples from "samsum"
    print(dash_line)
    print('EXAMPLE ', i + 1)
    print(dash_line)
    print('DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

---------------------------------------------------------------------------------------------------
EXAMPLE  1
---------------------------------------------------------------------------------------------------
DIALOGUE:
Nick: You look absolutely gorgeous and have a lovely smile. 
Nick: Would love to get to know you a bit more. How about we meet up for a drink sometime?
Jane: Hmmm... You're shooting a bit above your range aren't you?
Nick: Why would you think that hon?
Jane: Because I'm not that desperate.
Nick: That was a bit below the belt.
Nick: You're nice but you're not THAT hot.
Jane: Oh is your poor little dick shriveling at the thought?
Nick: Actually I'll take it back. Forget about the drink.
Nick: Forget I ever wrote to you.
Jane: Bye loser!
Nick: Fucking bitch!
Jane: You're welcome!
---------------------------------------------------------------------------------------------------
HUMAN SUMMARY:
Nick finds Jane pretty and invites her for a drink to get to know her better. Ja

**3. FLAN-T5 model**

In [5]:
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name) # downloading LLM model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True) # to convert raw text from conversation into vectors

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

**4. Text summarization without prompt engineering**

In [7]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = dialogue

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('EXAMPLE ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n') # this approach generates poor output

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Nick: You look absolutely gorgeous and have a lovely smile. 
Nick: Would love to get to know you a bit more. How about we meet up for a drink sometime?
Jane: Hmmm... You're shooting a bit above your range aren't you?
Nick: Why would you think that hon?
Jane: Because I'm not that desperate.
Nick: That was a bit below the belt.
Nick: You're nice but you're not THAT hot.
Jane: Oh is your poor little dick shriveling at the thought?
Nick: Actually I'll take it back. Forget about the drink.
Nick: Forget I ever wrote to you.
Jane: Bye loser!
Nick: Fucking bitch!
Jane: You're welcome!
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Nick finds Jane pretty and invites her for a drink to get to know h

**5. Text summarization with prompt engineering** **- zero shot**

In [9]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
    """

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('EXAMPLE ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITH PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
EXAMPLE  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Nick: You look absolutely gorgeous and have a lovely smile. 
Nick: Would love to get to know you a bit more. How about we meet up for a drink sometime?
Jane: Hmmm... You're shooting a bit above your range aren't you?
Nick: Why would you think that hon?
Jane: Because I'm not that desperate.
Nick: That was a bit below the belt.
Nick: You're nice but you're not THAT hot.
Jane: Oh is your poor little dick shriveling at the thought?
Nick: Actually I'll take it back. Forget about the drink.
Nick: Forget I ever wrote to you.
Jane: Bye loser!
Nick: Fucking bitch!
Jane: You're welcome!
---------------------------------------------------------------------------------------------------
SUMMARY:
Nick finds Jane pretty and invites her for a drink to get to know her better. Jane

**6. Text summarization with prompt engineering** **- one shot**

In [14]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""

    dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""

    return prompt

example_indices_full = [50]
example_index_to_summarize = 100

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)

def make_call(prompt, no):
    summary = dataset['test'][example_index_to_summarize]['summary']

    inputs = tokenizer(one_shot_prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print(f'HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION - {no} SHOT:\n{output}')

make_call(one_shot_prompt, 1)


Dialogue:

Nick: You look absolutely gorgeous and have a lovely smile. 
Nick: Would love to get to know you a bit more. How about we meet up for a drink sometime?
Jane: Hmmm... You're shooting a bit above your range aren't you?
Nick: Why would you think that hon?
Jane: Because I'm not that desperate.
Nick: That was a bit below the belt.
Nick: You're nice but you're not THAT hot.
Jane: Oh is your poor little dick shriveling at the thought?
Nick: Actually I'll take it back. Forget about the drink.
Nick: Forget I ever wrote to you.
Jane: Bye loser!
Nick: Fucking bitch!
Jane: You're welcome!

What was going on?
Nick finds Jane pretty and invites her for a drink to get to know her better. Jane rejects Nick and is unpleasant to him. Nick suggests Jane to forget about their conversation.



Dialogue:

Chris: Hi there! Where are you? Any chance of skyping?
Rick: Hi! Our last two days in Cancun before flying to Havana. Yeah, skyping is an idea. When would it suit you?
Rick: We don't have the b

**7. Text summarization with prompt engineering - three shot**

In [16]:
example_indices_full = [50,60,70]
example_index_to_summarize = 100

three_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(three_shot_prompt)
make_call(one_shot_prompt, 3)


Dialogue:

Nick: You look absolutely gorgeous and have a lovely smile. 
Nick: Would love to get to know you a bit more. How about we meet up for a drink sometime?
Jane: Hmmm... You're shooting a bit above your range aren't you?
Nick: Why would you think that hon?
Jane: Because I'm not that desperate.
Nick: That was a bit below the belt.
Nick: You're nice but you're not THAT hot.
Jane: Oh is your poor little dick shriveling at the thought?
Nick: Actually I'll take it back. Forget about the drink.
Nick: Forget I ever wrote to you.
Jane: Bye loser!
Nick: Fucking bitch!
Jane: You're welcome!

What was going on?
Nick finds Jane pretty and invites her for a drink to get to know her better. Jane rejects Nick and is unpleasant to him. Nick suggests Jane to forget about their conversation.



Dialogue:

Salma: <file_video>
Salma: the latest cat meme
Hugh: oh sweet, I can never get enough of those lol
Hugh: <file_photo>
Salma: hahaha same

What was going on?
Salma and Hugh like cat memes.



Di

**8. Generative configuration**

In [18]:
generation_config1 = GenerationConfig(max_new_tokens=50)
generation_config2= GenerationConfig(max_new_tokens=10, do_sample=True, temperature=1.0)
generation_config3 = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
generation_config4 = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
generation_config5 = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

def make_config(config):
    inputs = tokenizer(three_shot_prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            generation_config=config,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print(f'MODEL GENERATION - FEW SHOT:\n{output}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

print(f'MAX NEW TOKENS: 50, TEMPERATURE: 1.0\n')
make_config(generation_config5)
print(f'MAX NEW TOKENS: 10, TEMPERATURE: 1.0\n')
make_config(generation_config2)
print(f'MAX NEW TOKENS: 50, TEMPERATURE: 0.1\n')
make_config(generation_config3)


MAX NEW TOKENS: 50, TEMPERATURE: 1.0

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
Chris and Rick are going to skype tomorrow at 3pm in Rick's place. Chris cannot use Internet in Cuba. Rick will take care of Hughes.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Sam won't finish work till 5. Sam is bringing him over about 9 am. Sam will see Abdellilah in the morning. 

MAX NEW TOKENS: 10, TEMPERATURE: 1.0

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
Rick is in Cancun today. They are
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Sam won't finish work till 5. Sam is bringing him over about 9 am. Sam will see Abdellilah in the morning. 

MAX NEW TOKENS: 50, TEMPERATURE: 